In [ ]:
class Pretrain3TSeriesTransformer(nn.Module):
    """ Vision Transformer with support for patch or hybrid CNN input stage"""

    def __init__(self,
                 encoder: TSeriesTransformer,
                 head_num_layers=5,
                 mlp_dim=2048,
                 pretrained_cfg=None,):
        super().__init__()
        self.encoder = encoder
        self.real_head = build_mlp(num_layers=3, input_dim=encoder.embed_dim, mlp_dim=mlp_dim, output_dim=encoder.series_length // 2 + 1)
        self.imag_head = build_mlp(num_layers=3, input_dim=encoder.embed_dim, mlp_dim=mlp_dim, output_dim=encoder.series_length // 2 + 1)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward(self, x, mask=None):
        x = self.encoder(x, mask)
        real, imag = self.real_head(x), self.imag_head(x)
        freq = torch.complex(real.type(torch.float), imag.type(torch.float))
        time_series = torch.fft.irfft(freq)
        return time_series

